In [189]:
import pandas as pd
import numpy as np

In [190]:
df = pd.read_csv('TD_HOSPITAL_TRAIN.csv')

In [191]:
df.drop(['dose','pdeath'],axis=1)
for index in range(len(df['sex'])):
    val = df['sex'][index]
    if val == "male" or val == "Male" or val == "M" or val == 1:
        df['sex'][index] = 0
    else:
        df['sex'][index] = 1

# print(df['sex'])

def attempt_convert_to_float(column):
    try:
        return column.astype(float)
    except ValueError:  # If conversion fails, return the original column
        return column

# Apply the function to each column
df = df.apply(attempt_convert_to_float)

for col in df.columns:
    if df.dtypes[col] == "float64":
        df[col].fillna(df[col].mean(), inplace=True)

# df = pd.get_dummies(df, columns=['sex', 'race', 'primary'], drop_first=True)
df = pd.get_dummies(df, columns=['dnr','race', 'primary', 'disability', 'income' ,'extraprimary', 'cancer' ], drop_first=True)

df.to_csv('output.csv')

/var/folders/6w/qt75n21508bdvzq5z1lgl6sr0000gn/T/ipykernel_8802/3518595214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'][index] = 0
/var/folders/6w/qt75n21508bdvzq5z1lgl6sr0000gn/T/ipykernel_8802/3518595214.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'][index] = 1


In [192]:

correlation = df.corr()['death'].sort_values(ascending=False)
# print(correlation)
# threshold = float(input("Enter the threshold value you would like to analyze:"))
# print(threshold)

threshold = 0.05

# Identify features that have a correlation magnitude above the threshold
significant_features = correlation[correlation.abs() > threshold].index.tolist()

# Exclude the target variable 'death' from the features list
significant_features.remove('death')

new_df = df[significant_features + ['death']]
print(new_df)

new_df.to_csv('sig.csv')


      extraprimary_Cancer  primary_Lung Cancer    psych6  \
0                       0                    0  2.000000   
1                       1                    0  1.000000   
2                       0                    0  0.000000   
3                       0                    0  1.839000   
4                       0                    0  6.000000   
...                   ...                  ...       ...   
7053                    0                    0  5.000000   
7054                    0                    0  0.000000   
7055                    0                    0  0.000000   
7056                    0                    0  2.317871   
7057                    0                    0  5.000000   

      primary_MOSF w/Malig    psych2       age  psych5  primary_Colon Cancer  \
0                        0  2.000000  76.56396    30.0                     0   
1                        0  1.000000  63.33499    11.5                     1   
2                        0  0.000000  7

In [193]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

df = pd.read_csv('sig.csv')

# Split data into features and target variable
X = df.drop('death', axis=1)
y = df['death']

# Standardize numerical features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, 
                    epochs=20, 
                    batch_size=32, 
                    validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Sample prediction
sample_data = X_test.iloc[0]  # get the first row from the test set
predicted_prob = model.predict(np.array([sample_data]))[0][0]
print(f"The probability of death is: {predicted_prob * 100:.2f}%")
predicted_label = (predicted_prob > 0.5).astype(int)
print(f"Predicted Label: {predicted_label}")



Epoch 1/20
177/177 [==============================] - 0s 1ms/step - loss: 0.4714 - accuracy: 0.7779 - val_loss: 0.3727 - val_accuracy: 0.8272
Epoch 2/20
177/177 [==============================] - 0s 999us/step - loss: 0.3771 - accuracy: 0.8344 - val_loss: 0.3524 - val_accuracy: 0.8322
Epoch 3/20
177/177 [==============================] - 0s 805us/step - loss: 0.3549 - accuracy: 0.8425 - val_loss: 0.3384 - val_accuracy: 0.8371
Epoch 4/20
177/177 [==============================] - 0s 929us/step - loss: 0.3389 - accuracy: 0.8480 - val_loss: 0.3263 - val_accuracy: 0.8399
Epoch 5/20
177/177 [==============================] - 0s 723us/step - loss: 0.3255 - accuracy: 0.8537 - val_loss: 0.3224 - val_accuracy: 0.8463
Epoch 6/20
177/177 [==============================] - 0s 746us/step - loss: 0.3181 - accuracy: 0.8537 - val_loss: 0.3154 - val_accuracy: 0.8428
Epoch 7/20
177/177 [==============================] - 0s 778us/step - loss: 0.3086 - accuracy: 0.8604 - val_loss: 0.3096 - val_accuracy: 0